In [ ]:
from gensim import corpora, models
import re
# 불용어 목록 정의
stop_words = {'수', '이', '그', '등', '더', '만', '들', '것', '같', '되', '있', '한', '티르티르', 'pzp', '있어요'}

# 텍스트 전처리 함수
def preprocess_text(text):
    text = re.sub(r'\[.*?\]', '', text)  # 대괄호 내용 제거
    text = re.sub(r'\W+', ' ', text)  # 특수 문자 제거
    tokens = text.lower().split()  # 공백 기준으로 토큰화
    tokens = [word for word in tokens if word not in stop_words]  # 불용어 제거
    return tokens

# `content` 컬럼 전처리
data['processed'] = data['content'].apply(preprocess_text)

# 말뭉치 및 사전 생성
dictionary = corpora.Dictionary(data['processed'])
corpus = [dictionary.doc2bow(text) for text in data['processed']]

# LDA 모델 학습
lda_model = models.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=2)

# 토픽 출력
topics = lda_model.print_topics(num_words=5)
for idx, topic in topics:
    print(f"토픽 {idx}: {topic}")

In [ ]:
import re
import os
import pandas as pd
from gensim import corpora, models
from nltk.tokenize import RegexpTokenizer

# 기본 경로 설정
base_path = r"C:\Users\SJ\Desktop\새싹_종로\파이썬"
stopwords_path = os.path.join(base_path, "stopwords.txt")

# 불용어 파일 읽기 함수
def load_stopwords(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            stop_words = set(line.strip() for line in f)
        return stop_words
    except FileNotFoundError:
        print(f"불용어 파일이 존재하지 않습니다: {file_path}")
        return set()

# 텍스트 전처리 함수 (간단한 정규식 기반 토크나이저 사용)
def preprocess_text(text, stopwords_path):
    stop_words = load_stopwords(stopwords_path)  # 최신 불용어 목록 로드
    tokenizer = RegexpTokenizer(r'\w+')  # 단어 기준 토크나이저
    text = re.sub(r'\[.*?\]', '', text)  # 대괄호 내용 제거
    tokens = tokenizer.tokenize(text)  # 텍스트를 단어 단위로 토크나이징
    tokens = [word for word in tokens if word not in stop_words]  # 불용어 제거
    return tokens

# LDA 분석 함수
def lda_analysis(file_name, stopwords_path, num_topics=5, passes=10):
    try:
        # 파일 경로 설정
        file_path = os.path.join(base_path, file_name)
        
        # 데이터 로드
        data = pd.read_csv(file_path, encoding='utf-8', on_bad_lines='skip', engine='python')
        print(f"파일 로드 성공: {file_name}")

        # 데이터 컬럼 이름을 소문자로 변환
        data.columns = data.columns.str.lower()

        # 'description' 컬럼 전처리
        data['description'] = data['description'].fillna('')  # 결측값 처리
        data['processed'] = data['description'].apply(lambda text: preprocess_text(text, stopwords_path))

        # 말뭉치 및 사전 생성
        dictionary = corpora.Dictionary(data['processed'])
        corpus = [dictionary.doc2bow(text) for text in data['processed']]

        # LDA 모델 학습
        lda_model = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=passes)

        # 토픽 출력
        print(f"\n=== {file_name}의 토픽 ===")
        topics = lda_model.print_topics(num_words=5)
        for idx, topic in topics:
            print(f"토픽 {idx}: {topic}")
    except FileNotFoundError as fnf_error:
        print(f"파일을 찾을 수 없습니다: {fnf_error}")
    except Exception as e:
        print(f"{file_name} 처리 중 오류 발생: {e}")

# 5개년 데이터 처리
file_names = [f"articles_{year}.csv" for year in range(2020, 2025)]
for file_name in file_names:
    lda_analysis(file_name, stopwords_path)
